# Model neuron simulation
## An analysis of learning performance changes in spiking neural networks(SNN)
### 김용주, 김태호

* HH model cell을 실험 조건에 맞게 배치해 볼 것
    * Ball And Stick Tutorial에서는 z축을 중심으로 2Pi/n rad마다 배치하였음.
    * 해당 실험에서는 n by n의 excitatory neurons를 z=0에 배치, n by n (n=10, 15, 20)의 inhibitory neurons를 z=-1에 배치해 보고자 함
    * excitatory neuron -> inhibitory neurons는 x 좌표와 y좌표가 둘 다 맞을 때 연결
    * inhibitory neuron -> excitatory neurons는 x 좌표와 y좌표가 둘 다 맞을 때를 제외하고 연결

* Q1 : input stimulation은 어떻게 구현할 수 있을까?
    * Convolutional한 MNIST 데이터셋으로 어떻게 excitatory neurons을 흥분시킬 수 있는가? -> NetCon을 이용하여 Value 에서 Frequency로 바꾸는 작업이 필요할 듯
* Q2 : 신경망의 가중치 설정 및 조작
    * CNN perceptron -> SNN neuron
        * Weight -> Synaptic Weight (Weight term)
        * Transfer function 에 대응되는 개념이 SNN에는 존재하지 않는가?
        * Refractory는 불변의 scale인가?
* Q3 : model image의 learning을 어떤 식으로 진행시킬 수 있을까?
    * STDP

## Class 선언

### Cell class
* neuron의 기본적인 position, 구성요소 등을 정의하는 함수
* 후술할 ExcitatoryCell과 InhibitoryCell에서 define되는 _setup_morphology()와 _setup_biophysics() 를 __init__에서 recall함으로써 neuron을 define함

### ExcitatoryCell
* 흥분성 뉴런을 정의하는 class
    * 휴지기 전압 : -65mV
    * 초기화 전압 : -65mV
    * Threshold : -52mV
    * Refractory period : 5ms
    
### InhibitoryCell
* 억제성 뉴런을 정의하는 class
    * 휴지기 전압 : -60mV
    * 초기화 전압 : -45mV
    * Threshold : -40mV
    * Refractory period : 2ms
    
* Refractory period는 설정 완료
    * 어떻게 휴지기 전압, 초기화 전압, Threshold를 설정 가능한가?
        * 휴지기 전압, 초기화 전압의 차이점은?

In [1]:
from neuron import h, gui
from neuron.units import ms, mV
h.load_file('stdrun.hoc')

1.0

In [2]:
from neuron import h
class Cell:
    def __init__(self, gid, x, y, z):
        self._gid = gid
        self._setup_morphology()
        self.all = self.soma.wholetree()
        self._setup_biophysics()
        self.x = self.y = self.z = 0
        h.define_shape()
        self._set_position(x, y, z)
        self._spike_detector = h.NetCon(self.soma(0.5)._ref_v, None, sec=self.soma)
        self.spike_times = h.Vector()
        self._spike_detector.record(self.spike_times)
        self._ncs = []
        self.soma_v = h.Vector().record(self.soma(0.5)._ref_v)

    def __repr__(self):
        return '{}[{}]'.format(self.name, self._gid)
    
    def _set_position(self, x, y, z):
        for sec in self.all:
            for i in range(sec.n3d()):
                sec.pt3dchange(i, x - self.x + sec.x3d(i), y - self.y + sec.y3d(i), z - self.z + sec.z3d(i), sec.diam3d(i))
        self.x, self.y, self.z = x, y, z

In [3]:
class ExcitatoryCell(Cell):
    name = 'ExcitatoryCell'
    
    def _setup_morphology(self):
        self.soma = h.Section(name='soma', cell=self)
        self.dend = h.Section(name='dend', cell=self)
        self.dend.connect(self.soma)
        self.soma.L = self.soma.diam = 12.6157
        self.dend.L = 200
        self.dend.diam = 1

    def _setup_biophysics(self):
        for sec in self.all:
            sec.Ra = 100    # Axial resistance in Ohm * cm
            sec.cm = 1      # Membrane capacitance in micro Farads / cm^2
        self.soma.insert('hh')                                          
        for seg in self.soma:
            seg.hh.gnabar = 0.12  # Sodium conductance in S/cm2
            seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2
            seg.hh.gl = 0.0003    # Leak conductance in S/cm2
            seg.hh.el = -54.3     # Reversal potential in mV
            
        self.dend.insert('pas')                 
        for seg in self.dend:
            seg.pas.g = 0.001  # Passive conductance in S/cm2
            seg.pas.e = -65    # Leak reversal potential mV
        self.syn = h.ExpSyn(self.dend(0.5))
        self.syn.tau = 2 * ms
        self.refrac = 5 * ms
        # input이 .syn이라는 한 지역을 통해 입력 되는 것으로 간주시킴.
"""
h.ExpSyn decay에 의해 두 개의 ExpSyn object가 같은 point에 있는 것이나 
linearly하게 더해지는 서로 다른 두 synapse가 한 군데에 input하는 것과 다름이 없다.
이 point를 dend(0.5)로 정하자.
"""
pass

In [4]:
class InhibitoryCell(Cell):
    name = 'InhibitoryCell'
    
    def _setup_morphology(self):
        self.soma = h.Section(name='soma', cell=self)
        self.dend = h.Section(name='dend', cell=self)
        self.dend.connect(self.soma)
        self.soma.L = self.soma.diam = 12.6157
        self.dend.L = 200
        self.dend.diam = 1

    def _setup_biophysics(self):
        for sec in self.all:
            sec.Ra = 100    # Axial resistance in Ohm * cm
            sec.cm = 1      # Membrane capacitance in micro Farads / cm^2
        self.soma.insert('hh')                                          
        for seg in self.soma:
            seg.hh.gnabar = 0.12  # Sodium conductance in S/cm2
            seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2
            seg.hh.gl = 0.0003    # Leak conductance in S/cm2
            seg.hh.el = -54.3     # Reversal potential in mV
            
        self.dend.insert('pas')                 
        for seg in self.dend:
            seg.pas.g = 0.001  # Passive conductance in S/cm2
            seg.pas.e = -60    # Leak reversal potential mV
        self.syn = h.ExpSyn(self.dend(0.5))
        self.syn.tau = 2 * ms
        self.refrac = 2*ms
        # input이 .syn이라는 한 지역을 통해 입력 되는 것으로 간주시킴.
"""
h.ExpSyn decay에 의해 두 개의 ExpSyn object가 같은 point에 있는 것이나 
linearly하게 더해지는 서로 다른 두 synapse가 한 군데에 input하는 것과 다름이 없다.
이 point를 dend(0.5)로 정하자.
"""
pass

In [5]:
class ExSquare:
    def __init__(self, N=5, stim_w=0.04, stim_t=9, stim_delay=1, syn_w=0.01, syn_delay=5, d=25):
        self._syn_w = syn_w  #Stimulus weight
        self._syn_delay = syn_delay
        self._create_cells(N, d)
        self._netstim = h.NetStim()
        self._netstim.number = 1
        self._netstim.start = stim_t
        self._nc = h.NetCon(self._netstim, self.cells[0].syn)
        self._nc.delay = stim_delay
        self._nc.weight[0] = stim_w
        
    def _create_cells(self, N, d):
        self.cells = []
        for i in range(N):
            for j in range(N):
                self.cells.append(ExcitatoryCell(100*i+j, d * i, d * j, 0))
    """
    Ball And Stick #2의 create_n_BallAndStick 함수와 같은 방식으로 세포를 선언/배치.
    단, 세포들이 return되는 create_n_BallAndStick과 달리 self.cells에 저장됨.
    """
    def _connect_cells(self, insquare):
        for source, target in zip(self.cells, insquare.cells):
            nc = h.NetCon(source.soma(0.5)._ref_v, target.syn, sec=source.soma)
            nc.weight[0] = self._syn_w
            nc.delay = self._syn_delay
            source._ncs.append(nc)

In [6]:
class InSquare:
    def __init__(self, N=5, stim_w=0.04, stim_t=9, stim_delay=1, syn_w=-0.01, syn_delay=5, d=25):
        self._syn_w = syn_w  #Stimulus weight
        self._syn_delay = syn_delay
        self._create_cells(N, d)
        self._netstim = h.NetStim()
        self._netstim.number = 1
        self._netstim.start = stim_t
        self._nc = h.NetCon(self._netstim, self.cells[0].syn)
        self._nc.delay = stim_delay
        self._nc.weight[0] = stim_w
        
    def _create_cells(self, N, d):
        self.cells = []
        for i in range(N):
            for j in range(N):
                self.cells.append(InhibitoryCell(-100*i-j, d * i, d * j, -50))
    """
    Ball And Stick #2의 create_n_BallAndStick 함수와 같은 방식으로 세포를 선언/배치.
    단, 세포들이 return되는 create_n_BallAndStick과 달리 self.cells에 저장됨.
    """
    def _connect_cells(self, exsquare):
        
        for cell in self.cells:
            for excell in exsquare.cells:
                if self.cells.index(cell) != exsquare.cells.index(excell):
                    source = cell
                    target = excell
                    nc = h.NetCon(source.soma(0.5)._ref_v, target.syn, sec=source.soma)
                    nc.weight[0] = self._syn_w
                    nc.delay = self._syn_delay
                    source._ncs.append(nc)

In [7]:
n = int(input())
exsquare = ExSquare(N=n)
insquare = InSquare(N=n)
exsquare._connect_cells(insquare)
insquare._connect_cells(exsquare)

15


In [8]:
'''
Excitatory Cell과 Inhibitory Cell의 NetCon Synapse 수가 
각각 1, (n^2)-1임을 확인할 수 있다.
(One-to-one, All-to-all except coupled neuron을 만족)
'''

print(len(exsquare.cells[0]._ncs))
print(len(insquare.cells[0]._ncs))


1
224


In [9]:
shape_window = h.PlotShape(True)
shape_window.show(0)

1.0

## MNIST loading

In [43]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd

import tensorflow_datasets as tfds

In [11]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

In [12]:
ds = tfds.load('mnist', split='train', shuffle_files=True)
assert isinstance(ds, tf.data.Dataset)
print(ds)

<PrefetchDataset element_spec={'image': TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>


In [17]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [18]:
x, y = mnist['data'], mnist['target']
print(x.shape, y.shape)

(70000, 784) (70000,)


In [48]:
x.loc[0, :]

pixel1      0.0
pixel2      0.0
pixel3      0.0
pixel4      0.0
pixel5      0.0
           ... 
pixel780    0.0
pixel781    0.0
pixel782    0.0
pixel783    0.0
pixel784    0.0
Name: 0, Length: 784, dtype: float64

In [44]:
digit = x.loc[0,:]
digit_image = digit.reshape(28, 28)
print(y[0])
plt.imshow(digit_image, cmap='binary')
plt.axis('off')
plt.show()

AttributeError: 'Series' object has no attribute 'reshape'